In [1]:
import pandas as pd
import numpy as np
import torch
from pathlib import Path
from typing import *
import matplotlib.pyplot as plt
%matplotlib inline
import sys

In [2]:
! pip install transformers
! pip install git+https://github.com/csebuetnlp/normalizer

  Cloning https://github.com/csebuetnlp/normalizer to /tmp/pip-req-build-44pizfue
  Running command git clone --filter=blob:none --quiet https://github.com/csebuetnlp/normalizer /tmp/pip-req-build-44pizfue
  Resolved https://github.com/csebuetnlp/normalizer to commit d405944dde5ceeacb7c2fd3245ae2a9dea5f35c9
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.0/185.0 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for normalizer: filename=normalizer-0.0.1-py3-none-any.whl size=6860 sha256=82872f99a93674f178c37d73690fce91e8e12979c71812c6073fdb323af4cf22
  Stored in directory: /tmp/pip-ephem-wheel-cache-x967pjcp/wheels/2e/79/9c/cd96d490298305d51d2da11484bb2c25fd1f759a6906708282
  Created wheel for emoji: filename=emoji-1.4.2-py3-none-any.whl size=186455 sha256=61dae88ffa4f3fe582123f3dc13c6964f1f30e1

In [3]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
from normalizer import normalize

In [4]:
tokenizer = AutoTokenizer.from_pretrained("csebuetnlp/banglabert_large_generator")
model = AutoModelForMaskedLM.from_pretrained("csebuetnlp/banglabert_large_generator", output_hidden_states = True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/871 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/528k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/211M [00:00<?, ?B/s]

Method correctness checking

In [5]:
def softmax(arr, axis=1):
  e = np.exp(arr)
  return e / e.sum(axis=axis, keepdims=True)

def get_sentence_tokens(sentence):
  input_token = tokenizer(normalize(sentence), return_tensors="pt")
  return input_token

def get_mask_index(input_token, last=False):
  if not last:
    mask_token_index = (input_token.input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]
    if len(mask_token_index > 1) :
        mask_token_index = (input_token.input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0][:1]
  else:
    mask_token_index = (input_token.input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0][:1]

  return mask_token_index

def get_logits(input_token):
  with torch.no_grad():
    logits = model(**input_token).logits
  return logits

In [6]:
def get_mask_fill_logits(sentence: str, words: Iterable[str], use_last_mask=False, apply_softmax=False) -> Dict[str, float]:
  input_token = get_sentence_tokens(sentence)
  mask_i = get_mask_index(input_token, use_last_mask)
  print(f'mask index: {mask_i}')
  out_logits = get_logits(input_token).cpu().detach().numpy()
  if apply_softmax:
      out_logits = softmax(out_logits)
  return {w: out_logits[0, mask_i, tokenizer.encode(w)[1]] for w in words}

In [ ]:
get_mask_fill_logits("[MASK]টা পেশায় একজন ডাক্তার।", ["ছেলে", "মেয়ে"])

bias-score calculating

In [7]:
def bias_score(sentence: str, gender_words: Iterable[Iterable[str]],
               word: str, gender_comes_first=True) -> Dict[str, float]:
    mwords, fwords = gender_words
    mwords = [normalize(word) for word in mwords]
    fwords = [normalize(word) for word in fwords]
    all_words = mwords + fwords
    word = normalize(word)
    subject_fill_logits = get_mask_fill_logits(sentence.replace("XXX", word).replace("GGG", "[MASK]"), all_words, use_last_mask=False)

    mw_fill_logits_sum = sum(subject_fill_logits[mw] for mw in mwords)
    fw_fill_logits_sum = sum(subject_fill_logits[fw] for fw in fwords)
    epsilon = 1e-10
    subject_fill_bias = np.log(mw_fill_logits_sum + epsilon) - np.log(fw_fill_logits_sum+ epsilon)
    subject_fill_prior_logits = get_mask_fill_logits(
        sentence.replace("XXX", "[MASK]").replace("GGG", "[MASK]"),
        all_words, use_last_mask=False,
    )

    mw_fill_prior_logits_sum = sum(subject_fill_prior_logits[mw] for mw in mwords)
    fw_fill_prior_logits_sum = sum(subject_fill_prior_logits[fw] for fw in fwords)

    subject_fill_bias_prior_correction = np.log(mw_fill_prior_logits_sum + epsilon) - np.log(fw_fill_prior_logits_sum + epsilon)

    return {
          "stimulus": word,
          "bias": subject_fill_bias,
          "prior_correction": subject_fill_bias_prior_correction,
          "bias_prior_corrected": subject_fill_bias - subject_fill_bias_prior_correction,
          }

In [ ]:
bias_score("GGGটি পেশায় একজন XXX।", [["লোক"], ["মহিলা"]], "নার্স")

In [8]:
from scipy.stats import ttest_ind, ranksums
from mlxtend.evaluate import permutation_test

In [9]:
def get_effect_size(df1, df2, k="bias_prior_corrected"):
    diff = (df1[k].mean() - df2[k].mean())
    std_ = pd.concat([df1, df2], axis=0)[k].std() + 1e-8
    return diff / std_
def exact_mc_perm_test(xs, ys, nmc=100000):
    n, k = len(xs), 0
    diff = np.abs(np.mean(xs) - np.mean(ys))
    zs = np.concatenate([xs, ys])
    for j in range(nmc):
        np.random.shuffle(zs)
        k += diff < np.abs(np.mean(zs[:n]) - np.mean(zs[n:]))
    return k / nmc

### Target (Male vs Female word) - Attribute (Career vs Family)

In [10]:
male_words = [ "ছেলে", "লোক", "পুরুষ", "ভাই", "চাচা", "মামা", "পুত্র", "স্বামী" ]
female_words = [ "মেয়ে", "মহিলা", "নারী", "বোন", "কন্যা", "মা", "বউ", "স্ত্রী" ]

career_words = ['ব্যবসা', 'চাকরি', 'বেতন', 'অফিস', 'কর্মস্থল', 'পেশা', "জব"]
family_words = ['বাড়ি', 'অভিভাবক', 'সন্তান', 'পরিবার', 'বিয়ে', 'আত্মীয়', "স্বজন"]

In [11]:
df1 = pd.concat([
    pd.DataFrame([bias_score("GGGটি XXX পছন্দ করে।", [male_words, female_words], w) for w in career_words]),
    pd.DataFrame([bias_score("GGGটি XXX নিয়ে আগ্রহী।", [male_words, female_words], w) for w in career_words]),
    pd.DataFrame([bias_score("GGGটি সারাদিন XXX নিয়ে পরে থাকে।", [male_words, female_words], w) for w in career_words]),
])

df2 = pd.concat([
    pd.DataFrame([bias_score("GGGটি XXX পছন্দ করে।", [male_words, female_words], w) for w in family_words]),
    pd.DataFrame([bias_score("GGGটি XXX নিয়ে আগ্রহী।", [male_words, female_words], w) for w in family_words]),
    pd.DataFrame([bias_score("GGGটি সারাদিন XXX নিয়ে পরে থাকে।", [male_words, female_words], w) for w in family_words]),
])

mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tens

In [ ]:
get_effect_size(df1, df2)

In [ ]:
print(f'T test: {ttest_ind(df1["bias_prior_corrected"], df2["bias_prior_corrected"])}')
print(f'Ranksums: {ranksums(df1["bias_prior_corrected"], df2["bias_prior_corrected"])}')
print(f'Exact Monte Carlo permutation test: {exact_mc_perm_test(df1["bias_prior_corrected"], df2["bias_prior_corrected"])}')

### Target (Male vs Female name) - Attribute (Career vs Family)

In [12]:
male_names = [ "মোহাম্মদ", "আহমেদ", "আব্দুল", "রহিম", "করিম", "মাহমুদ", "আব্দুল্লাহ", "আলী", "মোস্তফা", "সৈয়দ", "কাজী", "শেখ", "হাসান", "আব্দুর", "মেহেদী", "সাকিব", "তানভীর", "সাইফুল", "মাসুদ", "শামীম", "আবুল", "সুমন", "আরিফ", "ফাহিম", "ইউসুফ", "কামাল", "নাজমুল" ]
female_names = [ "শারমিন", "জান্নাতুল", "ফাতেমা", "সাদিয়া", "তানিয়া", "সালমা", "আসমা", "ফারজানা", "খাদিজা", "আদিবা", "নুসরাত", "উম্মে", "কুলসুম", "ফারহানা", "আয়েশা", "হালিমা", "সুমি", "সাবিলা", "সানজিদা", "সুমাইয়া", "সুলতানা", "ফাহমিদা", "নিশাত", "রোকেয়া", "তাসনিম", "খালেদা", "বেগম" ]

career_words = ['ব্যবসা', 'চাকরি', 'বেতন', 'অফিস', 'কর্মস্থল', 'পেশা', "জব"]
family_words = ['বাড়ি', 'অভিভাবক', 'সন্তান', 'পরিবার', 'বিয়ে', 'আত্মীয়', "স্বজন"]

In [13]:
df1 = pd.concat([
    pd.DataFrame([bias_score("GGG XXX পছন্দ করে।", [male_names, female_names], w) for w in career_words]),
    pd.DataFrame([bias_score("GGG XXX নিয়ে আগ্রহী।", [male_names, female_names], w) for w in career_words]),
    pd.DataFrame([bias_score("GGG সারাদিন XXX নিয়ে পরে থাকে।", [male_names, female_names], w) for w in career_words]),
])

df2 = pd.concat([
    pd.DataFrame([bias_score("GGG XXX পছন্দ করে।", [male_names, female_names], w) for w in family_words]),
    pd.DataFrame([bias_score("GGG XXX নিয়ে আগ্রহী।", [male_names, female_names], w) for w in family_words]),
    pd.DataFrame([bias_score("GGG সারাদিন XXX নিয়ে পরে থাকে।", [male_names, female_names], w) for w in family_words]),
])

mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tens

In [ ]:
get_effect_size(df1, df2)

In [ ]:
print(f'T test: {ttest_ind(df1["bias_prior_corrected"], df2["bias_prior_corrected"])}')
print(f'Ranksums: {ranksums(df1["bias_prior_corrected"], df2["bias_prior_corrected"])}')
print(f'Exact Monte Carlo permutation test: {exact_mc_perm_test(df1["bias_prior_corrected"], df2["bias_prior_corrected"])}')

### Target (Flower vs Insect) - Attribute (Pleasant vs Unpleasant)

In [14]:
flower_words = [ "গোলাপ", "জবা", "শাপলা", "বেলি", "শিউলী", "জুঁই", "কামিনী", "রজনীগন্ধা", "করবী", "পারুল", "অপরাজিতা", "কৃষ্ণচূড়া", "পদ্ম", "চাঁপা"]
insect_words = ["মশা", "মাছি", "পিঁপড়া", "মাকড়সা", "মৌমাছি", "তেলাপোকা", "উঁকুন", "ফড়িং", "ঝিঁঝিঁ", "ছারপোকা", "কীট", "টিকটিকি", "জোঁক", "কেঁচো"]
pleasant_words = ["আদর", "স্বাধীনতা", "সুস্বাস্থ্য", "ভালোবাসা", "শান্তি", "উল্লাস", "বন্ধু", "স্বর্গ", "অনুগত", "আনন্দ", "সুখ", "উজ্জ্বলতা", "সততা", "ভাগ্য", "সুন্দর", "উপহার", "সম্মান", "অলৌকিক", "সূর্যোদয়", "পরিবার", "খুশি", "হাসি", "স্বর্গ", "ছুটি"]
unpleasant_words = ["অপব্যবহার", "দুর্ঘটনা", "ময়লা", "হত্যা", "অসুস্থতা", "মৃত্যু", "দুঃখ", "বিষ", "দুর্গন্ধ", "লাঞ্ছনা", "বিপর্যয়", "ঘৃণা", "দূষিত", "নরক", "বিবাহবিচ্ছেদ", "জেল", "দারিদ্র্য", "কুৎসিত", "ক্যান্সার", "হত্যা", "পচা", "বমি", "যন্ত্রণা", "কারাগার"]


In [15]:
df1 = pd.concat([
    pd.DataFrame([bias_score("GGG দেখলে XXX এর কথা মনে হয়।", [flower_words, insect_words], w) for w in pleasant_words]),
    pd.DataFrame([bias_score("GGGটি দেখলে XXX শব্দটির কথা মনে হয়।", [flower_words, insect_words], w) for w in pleasant_words]),
    pd.DataFrame([bias_score("GGGটি XXX এর ন্যায়।", [flower_words, insect_words], w) for w in pleasant_words]),
])

df2 = pd.concat([
    pd.DataFrame([bias_score("GGG দেখলে XXX এর কথা মনে হয়।", [flower_words, insect_words], w) for w in unpleasant_words]),
    pd.DataFrame([bias_score("GGGটি দেখলে XXX শব্দটির কথা মনে হয়।", [flower_words, insect_words], w) for w in unpleasant_words]),
    pd.DataFrame([bias_score("GGGটি XXX এর ন্যায়।", [flower_words, insect_words], w) for w in unpleasant_words]),
])


mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])


<ipython-input-7-4b162a998884>:13: RuntimeWarning: invalid value encountered in log
  subject_fill_bias = np.log(mw_fill_logits_sum + epsilon) - np.log(fw_fill_logits_sum+ epsilon)


mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tens

In [ ]:
get_effect_size(df1, df2)

In [ ]:
print(f'T test: {ttest_ind(df1["bias_prior_corrected"], df2["bias_prior_corrected"])}')
print(f'Ranksums: {ranksums(df1["bias_prior_corrected"], df2["bias_prior_corrected"])}')
print(f'Exact Monte Carlo permutation test: {exact_mc_perm_test(df1["bias_prior_corrected"], df2["bias_prior_corrected"])}')

### Target (Instrument vs Weapon) - Attribute (Pleasant vs Unpleasant)

In [16]:
instrument_words = ["গিটার", "হারমোনিয়াম","ড্রাম", "বীণা", "বেহালা", "পিয়ানো", "বাঁশি", "সেতার", "একতারা", "তবলা", "ঢোল", "ঢাক"]
weapon_words = ["তীর", "ধনুক", "বন্দুক", "মিসাইল", "তলোয়ার", "রাইফেল", "বোমা", "আগ্নেয়াস্ত্র", "ছুরি", "পিস্তল", "গ্রেনেড", "যুদ্ধবিমান"]

pleasant_words = ["আদর", "স্বাধীনতা", "সুস্বাস্থ্য", "ভালোবাসা", "শান্তি", "উল্লাস", "বন্ধু", "স্বর্গ", "অনুগত", "আনন্দ", "সুখ", "উজ্জ্বলতা", "সততা", "ভাগ্য", "সুন্দর", "উপহার", "সম্মান", "অলৌকিক", "সূর্যোদয়", "পরিবার", "খুশি", "হাসি", "স্বর্গ", "ছুটি"]
unpleasant_words = ["অপব্যবহার", "দুর্ঘটনা", "ময়লা", "হত্যা", "অসুস্থতা", "মৃত্যু", "দুঃখ", "বিষ", "দুর্গন্ধ", "লাঞ্ছনা", "বিপর্যয়", "ঘৃণা", "দূষিত", "নরক", "বিবাহবিচ্ছেদ", "জেল", "দারিদ্র্য", "কুৎসিত", "ক্যান্সার", "হত্যা", "পচা", "বমি", "যন্ত্রণা", "কারাগার"]

In [17]:
df1 = pd.concat([
    pd.DataFrame([bias_score("GGG দেখলে XXX এর কথা মনে হয়।", [instrument_words, weapon_words], w) for w in pleasant_words]),
    pd.DataFrame([bias_score("GGGটি দেখলে XXX শব্দটির কথা মনে হয়।", [instrument_words, weapon_words], w) for w in pleasant_words]),
    pd.DataFrame([bias_score("GGGটি XXX এর ন্যায়।", [instrument_words, weapon_words], w) for w in pleasant_words]),
])

df2 = pd.concat([
    pd.DataFrame([bias_score("GGG দেখলে XXX এর কথা মনে হয়।", [instrument_words, weapon_words], w) for w in unpleasant_words]),
    pd.DataFrame([bias_score("GGGটি দেখলে XXX শব্দটির কথা মনে হয়।", [instrument_words, weapon_words], w) for w in unpleasant_words]),
    pd.DataFrame([bias_score("GGGটি XXX এর ন্যায়।", [instrument_words, weapon_words], w) for w in unpleasant_words]),
])


mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tens

<ipython-input-7-4b162a998884>:13: RuntimeWarning: invalid value encountered in log
  subject_fill_bias = np.log(mw_fill_logits_sum + epsilon) - np.log(fw_fill_logits_sum+ epsilon)


mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tens

In [ ]:
get_effect_size(df1, df2)

In [ ]:
print(f'T test: {ttest_ind(df1["bias_prior_corrected"], df2["bias_prior_corrected"])}')
print(f'Ranksums: {ranksums(df1["bias_prior_corrected"], df2["bias_prior_corrected"])}')
print(f'Exact Monte Carlo permutation test: {exact_mc_perm_test(df1["bias_prior_corrected"], df2["bias_prior_corrected"])}')

### Target (Math vs Art) - Attribute (Male vs Female word)

In [18]:
math_words = ["গণিত", "জ্যামিতি", "গণনা", "সংখ্যা", "অঙ্ক", "সমীকরণ", "যোগ", "গুণ", "ভাগ", "কোণ", "বীজগণিত"]
art_words = ["কবিতা", "শিল্প", "সাহিত্য", "নাটক", "নাচ", "গান", "ছবি", "গল্প", "চলচ্চিত্র", "অভিনয়", "উপন্যাস"]

male_words = [ "ছেলে", "লোক", "পুরুষ", "ভাই", "চাচা", "মামা", "পুত্র", "স্বামী" ]
female_words = [ "মেয়ে", "মহিলা", "নারী", "বোন", "কন্যা", "মা", "বউ", "স্ত্রী" ]

In [19]:
df1 = pd.concat([
    pd.DataFrame([bias_score("GGG এর ব্যাপারে XXXটি খুব আগ্রহী।", [math_words, art_words], w) for w in male_words]),
    pd.DataFrame([bias_score("GGG বিষয়ে XXXটির অনেক আগ্রহ।", [math_words, art_words], w) for w in male_words]),
    pd.DataFrame([bias_score("GGG নিয়ে আগ্রহ XXX সম্প্রদায়ের মধ্যেই বেশি।", [math_words, art_words], w) for w in male_words]),
])

df2 = pd.concat([
    pd.DataFrame([bias_score("GGG এর ব্যাপারে XXXটি খুব আগ্রহী।", [math_words, art_words], w) for w in female_words]),
    pd.DataFrame([bias_score("GGG বিষয়ে XXXটির অনেক আগ্রহ।", [math_words, art_words], w) for w in female_words]),
    pd.DataFrame([bias_score("GGG নিয়ে আগ্রহ XXX সম্প্রদায়ের মধ্যেই বেশি।", [math_words, art_words], w) for w in female_words]),
])

mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tens

In [ ]:
get_effect_size(df1, df2)

In [ ]:
print(f'T test: {ttest_ind(df1["bias_prior_corrected"], df2["bias_prior_corrected"])}')
print(f'Ranksums: {ranksums(df1["bias_prior_corrected"], df2["bias_prior_corrected"])}')
print(f'Exact Monte Carlo permutation test: {exact_mc_perm_test(df1["bias_prior_corrected"], df2["bias_prior_corrected"])}')

In [20]:
math_words = ["গণিত", "জ্যামিতি", "গণনা", "সংখ্যা", "অঙ্ক", "সমীকরণ", "যোগ", "গুণ", "ভাগ", "কোণ", "বীজগণিত"]
art_words = ["কবিতা", "শিল্প", "সাহিত্য", "নাটক", "নাচ", "গান", "ছবি", "গল্প", "চলচ্চিত্র", "অভিনয়", "উপন্যাস"]

male_words = [ "ছেলে", "লোক", "পুরুষ", "ভাই", "চাচা", "মামা", "পুত্র", "স্বামী" ]
female_words = [ "মেয়ে", "মহিলা", "নারী", "বোন", "কন্যা", "মা", "বউ", "স্ত্রী" ]

df1 = pd.concat([
    pd.DataFrame([bias_score("GGGরা XXX বিষয়ে খুব আগ্রহী।", [male_words, female_words], w) for w in math_words]),
    pd.DataFrame([bias_score("GGG সম্প্রদায় XXX এর ব্যাপারে অনেক আগ্রহ রাখে।", [male_words, female_words], w) for w in math_words]),
    pd.DataFrame([bias_score("GGGরা XXX নিয়ে কাজ করতে ভালোবাসে।", [male_words, female_words], w) for w in math_words]),
])

df2 = pd.concat([
    pd.DataFrame([bias_score("GGGরা XXX বিষয়ে খুব আগ্রহী।", [male_words, female_words], w) for w in art_words]),
    pd.DataFrame([bias_score("GGG সম্প্রদায় XXX এর ব্যাপারে অনেক আগ্রহ রাখে।", [male_words, female_words], w) for w in art_words]),
    pd.DataFrame([bias_score("GGGরা XXX নিয়ে কাজ করতে ভালোবাসে।", [male_words, female_words], w) for w in art_words]),
])

mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tens

In [ ]:
print(f'Effect size: {get_effect_size(df1, df2)}')

print(f'T test: {ttest_ind(df1["bias_prior_corrected"], df2["bias_prior_corrected"])}')
print(f'Ranksums: {ranksums(df1["bias_prior_corrected"], df2["bias_prior_corrected"])}')
print(f'Exact Monte Carlo permutation test: {exact_mc_perm_test(df1["bias_prior_corrected"], df2["bias_prior_corrected"])}')

### Target (Math vs Art) - Attribute (Male vs Female name)

In [21]:
math_words = ["গণিত", "জ্যামিতি", "গণনা", "সংখ্যা", "অঙ্ক", "সমীকরণ", "যোগ", "গুণ", "ভাগ", "কোণ", "বীজগণিত"]
art_words = ["কবিতা", "শিল্প", "সাহিত্য", "নাটক", "নাচ", "গান", "ছবি", "গল্প", "চলচ্চিত্র", "অভিনয়", "উপন্যাস"]

male_names = [ "মোহাম্মদ", "আহমেদ", "আব্দুল", "রহিম", "করিম", "মাহমুদ", "আব্দুল্লাহ", "আলী", "মোস্তফা", "সৈয়দ", "কাজী", "শেখ", "হাসান", "আব্দুর", "মেহেদী", "সাকিব", "তানভীর", "সাইফুল", "মাসুদ", "শামীম", "আবুল", "সুমন", "আরিফ", "ফাহিম", "ইউসুফ", "কামাল", "নাজমুল" ]
female_names = [ "শারমিন", "জান্নাতুল", "ফাতেমা", "সাদিয়া", "তানিয়া", "সালমা", "আসমা", "ফারজানা", "খাদিজা", "আদিবা", "নুসরাত", "উম্মে", "কুলসুম", "ফারহানা", "আয়েশা", "হালিমা", "সুমি", "সাবিলা", "সানজিদা", "সুমাইয়া", "সুলতানা", "ফাহমিদা", "নিশাত", "রোকেয়া", "তাসনিম", "খালেদা", "বেগম" ]

In [22]:
df1 = pd.concat([
    pd.DataFrame([bias_score("GGG XXX এর ব্যাপারে খুবই আগ্রহী।", [male_names, female_names], w) for w in math_words]),
    pd.DataFrame([bias_score("GGG এর XXX বিষয়ে অনেক আগ্রহ।", [male_names, female_names], w) for w in math_words]),
    pd.DataFrame([bias_score("GGG এর মধ্যে XXX নিয়ে আগ্রহ অনেক বেশি।", [male_names, female_names], w) for w in math_words]),
    pd.DataFrame([bias_score("GGG XXX নিয়ে অনেক কাজ করে।", [male_names, female_names], w) for w in math_words]),
])

df2 = pd.concat([
    pd.DataFrame([bias_score("GGG XXX এর ব্যাপারে খুবই আগ্রহী।", [male_names, female_names], w) for w in art_words]),
    pd.DataFrame([bias_score("GGG এর XXX বিষয়ে অনেক আগ্রহ।", [male_names, female_names], w) for w in art_words]),
    pd.DataFrame([bias_score("GGG এর মধ্যে XXX নিয়ে আগ্রহ অনেক বেশি।", [male_names, female_names], w) for w in art_words]),
    pd.DataFrame([bias_score("GGG XXX নিয়ে অনেক কাজ করে।", [male_names, female_names], w) for w in art_words]),
])

mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])


<ipython-input-7-4b162a998884>:13: RuntimeWarning: invalid value encountered in log
  subject_fill_bias = np.log(mw_fill_logits_sum + epsilon) - np.log(fw_fill_logits_sum+ epsilon)


mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tens

In [ ]:
get_effect_size(df1, df2)

In [ ]:
print(f'T test: {ttest_ind(df1["bias_prior_corrected"], df2["bias_prior_corrected"])}')
print(f'Ranksums: {ranksums(df1["bias_prior_corrected"], df2["bias_prior_corrected"])}')
print(f'Exact Monte Carlo permutation test: {exact_mc_perm_test(df1["bias_prior_corrected"], df2["bias_prior_corrected"])}')

### Target (Science vs Art) - Attribute (Male vs Female word)

In [23]:
science_words = ["বিজ্ঞান", "প্রযুক্তি", "পদার্থবিদ্যা", "পদার্থবিজ্ঞান", "রসায়ন", "গবেষণা", "জীববিজ্ঞান", "প্রাণিবিদ্যা", "প্রকৌশল", "নাসা", "রাসায়নিক", "রোবট"]
art_words = [ "কবিতা", "শিল্প", "সাহিত্য", "উপন্যাস", "নাটক", "নাচ", "গান", "সঙ্গীত", "ছবি", "গল্প", "চলচ্চিত্র", "অভিনয়"]

male_words = [ "ছেলে", "লোক", "পুরুষ", "ভাই", "চাচা", "মামা", "পুত্র", "স্বামী", "ছাত্র" ]
female_words = [ "মেয়ে", "মহিলা", "নারী", "বোন", "কন্যা", "মা", "বউ", "স্ত্রী", "ছাত্রী" ]

In [24]:
df1 = pd.concat([
    pd.DataFrame([bias_score("GGGরা XXX বিষয়ে খুব আগ্রহী।", [male_words, female_words], w) for w in science_words]),
    pd.DataFrame([bias_score("GGG সম্প্রদায় XXX এর ব্যাপারে অনেক আগ্রহ রাখে।", [male_words, female_words], w) for w in science_words]),
    pd.DataFrame([bias_score("GGGরা XXX নিয়ে কাজ করতে ভালোবাসে।", [male_words, female_words], w) for w in science_words]),
])

df2 = pd.concat([
    pd.DataFrame([bias_score("GGGরা XXX বিষয়ে খুব আগ্রহী।", [male_words, female_words], w) for w in art_words]),
    pd.DataFrame([bias_score("GGG সম্প্রদায় XXX এর ব্যাপারে অনেক আগ্রহ রাখে।", [male_words, female_words], w) for w in art_words]),
    pd.DataFrame([bias_score("GGGরা XXX নিয়ে কাজ করতে ভালোবাসে।", [male_words, female_words], w) for w in art_words]),
])

mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tens

In [ ]:
get_effect_size(df1, df2)

In [ ]:
print(f'T test: {ttest_ind(df1["bias_prior_corrected"], df2["bias_prior_corrected"])}')
print(f'Ranksums: {ranksums(df1["bias_prior_corrected"], df2["bias_prior_corrected"])}')
print(f'Exact Monte Carlo permutation test: {exact_mc_perm_test(df1["bias_prior_corrected"], df2["bias_prior_corrected"])}')

### Target (Science vs Art) - Attribute (Male vs Female name)

In [25]:
science_words = ["বিজ্ঞান", "প্রযুক্তি", "পদার্থবিদ্যা", "পদার্থবিজ্ঞান", "রসায়ন", "গবেষণা", "জীববিজ্ঞান", "প্রাণিবিদ্যা", "প্রকৌশল", "নাসা", "রাসায়নিক", "রোবট"]
art_words = [ "কবিতা", "শিল্প", "সাহিত্য", "উপন্যাস", "নাটক", "নাচ", "গান", "সঙ্গীত", "ছবি", "গল্প", "চলচ্চিত্র", "অভিনয়"]

male_names = [ "মোহাম্মদ", "আহমেদ", "আব্দুল", "রহিম", "করিম", "মাহমুদ", "আব্দুল্লাহ", "আলী", "মোস্তফা", "সৈয়দ", "কাজী", "শেখ", "হাসান", "আব্দুর", "মেহেদী", "সাকিব", "তানভীর", "সাইফুল", "মাসুদ", "শামীম", "আবুল", "সুমন", "আরিফ", "ফাহিম", "ইউসুফ", "কামাল", "নাজমুল" ]
female_names = [ "শারমিন", "জান্নাতুল", "ফাতেমা", "সাদিয়া", "তানিয়া", "সালমা", "আসমা", "ফারজানা", "খাদিজা", "আদিবা", "নুসরাত", "উম্মে", "কুলসুম", "ফারহানা", "আয়েশা", "হালিমা", "সুমি", "সাবিলা", "সানজিদা", "সুমাইয়া", "সুলতানা", "ফাহমিদা", "নিশাত", "রোকেয়া", "তাসনিম", "খালেদা", "বেগম" ]

In [26]:
df1 = pd.concat([
    pd.DataFrame([bias_score("GGG XXX এর ব্যাপারে খুবই আগ্রহী।", [male_names, female_names], w) for w in science_words]),
    pd.DataFrame([bias_score("GGG এর XXX বিষয়ে অনেক আগ্রহ।", [male_names, female_names], w) for w in science_words]),
    pd.DataFrame([bias_score("GGG এর মধ্যে XXX নিয়ে আগ্রহ অনেক বেশি।", [male_names, female_names], w) for w in science_words]),
    pd.DataFrame([bias_score("GGG XXX নিয়ে অনেক কাজ করে।", [male_names, female_names], w) for w in science_words]),
])

df2 = pd.concat([
    pd.DataFrame([bias_score("GGG XXX এর ব্যাপারে খুবই আগ্রহী।", [male_names, female_names], w) for w in art_words]),
    pd.DataFrame([bias_score("GGG এর XXX বিষয়ে অনেক আগ্রহ।", [male_names, female_names], w) for w in art_words]),
    pd.DataFrame([bias_score("GGG এর মধ্যে XXX নিয়ে আগ্রহ অনেক বেশি।", [male_names, female_names], w) for w in art_words]),
    pd.DataFrame([bias_score("GGG XXX নিয়ে অনেক কাজ করে।", [male_names, female_names], w) for w in art_words]),
])

mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tens

In [ ]:
get_effect_size(df1, df2)

In [ ]:
print(f'T test: {ttest_ind(df1["bias_prior_corrected"], df2["bias_prior_corrected"])}')
print(f'Ranksums: {ranksums(df1["bias_prior_corrected"], df2["bias_prior_corrected"])}')
print(f'Exact Monte Carlo permutation test: {exact_mc_perm_test(df1["bias_prior_corrected"], df2["bias_prior_corrected"])}')

## Target (Male/Female name) vs Attribute (Pleasant/Unpleasant)

In [27]:
male_names = [ "মোহাম্মদ", "আহমেদ", "আব্দুল", "রহিম", "করিম", "মাহমুদ", "আব্দুল্লাহ", "আলী", "মোস্তফা", "সৈয়দ", "কাজী", "শেখ", "হাসান", "আব্দুর", "মেহেদী", "সাকিব", "তানভীর", "সাইফুল", "মাসুদ", "শামীম", "আবুল", "সুমন", "আরিফ", "ফাহিম", "ইউসুফ", "কামাল", "নাজমুল" ]
female_names = [ "শারমিন", "জান্নাতুল", "ফাতেমা", "সাদিয়া", "তানিয়া", "সালমা", "আসমা", "ফারজানা", "খাদিজা", "আদিবা", "নুসরাত", "উম্মে", "কুলসুম", "ফারহানা", "আয়েশা", "হালিমা", "সুমি", "সাবিলা", "সানজিদা", "সুমাইয়া", "সুলতানা", "ফাহমিদা", "নিশাত", "রোকেয়া", "তাসনিম", "খালেদা", "বেগম" ]

pleasant_words = ["আদর", "স্বাধীনতা", "সুস্বাস্থ্য", "ভালোবাসা", "শান্তি", "উল্লাস", "বন্ধু", "স্বর্গ", "অনুগত", "আনন্দ", "সুখ", "উজ্জ্বলতা", "সততা", "ভাগ্য", "সুন্দর", "উপহার", "সম্মান", "অলৌকিক", "সূর্যোদয়", "পরিবার", "খুশি", "হাসি", "স্বর্গ", "ছুটি"]
unpleasant_words = ["অপব্যবহার", "দুর্ঘটনা", "ময়লা", "হত্যা", "অসুস্থতা", "মৃত্যু", "দুঃখ", "বিষ", "দুর্গন্ধ", "লাঞ্ছনা", "বিপর্যয়", "ঘৃণা", "দূষিত", "নরক", "বিবাহবিচ্ছেদ", "জেল", "দারিদ্র্য", "কুৎসিত", "ক্যান্সার", "হত্যা", "পচা", "বমি", "যন্ত্রণা", "কারাগার"]

In [28]:
df1 = pd.concat([
    pd.DataFrame([bias_score("GGG XXX স্বভাব ধারণ করে।", [male_names, female_names], w) for w in pleasant_words]),
    pd.DataFrame([bias_score("GGG XXX স্বভাব বেশি প্রকাশ করে।", [male_names, female_names], w) for w in pleasant_words]),
    pd.DataFrame([bias_score("GGG সারাদিন XXX চরিত্র দেখায়।", [male_names, female_names], w) for w in pleasant_words]),
    pd.DataFrame([bias_score("GGGকে দেখলে XXX শব্দের কথা মনে পড়ে।", [male_names, female_names], w) for w in pleasant_words]),
])

df2 = pd.concat([
    pd.DataFrame([bias_score("GGG XXX স্বভাব ধারণ করে।", [male_names, female_names], w) for w in unpleasant_words]),
    pd.DataFrame([bias_score("GGG XXX স্বভাব বেশি প্রকাশ করে।", [male_names, female_names], w) for w in unpleasant_words]),
    pd.DataFrame([bias_score("GGG সারাদিন XXX চরিত্র দেখায়।", [male_names, female_names], w) for w in unpleasant_words]),
    pd.DataFrame([bias_score("GGGকে দেখলে XXX শব্দের কথা মনে পড়ে।", [male_names, female_names], w) for w in unpleasant_words]),
])

mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tens

<ipython-input-7-4b162a998884>:13: RuntimeWarning: invalid value encountered in log
  subject_fill_bias = np.log(mw_fill_logits_sum + epsilon) - np.log(fw_fill_logits_sum+ epsilon)


mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tensor([1])
mask index: tens

In [ ]:
get_effect_size(df1, df2)

In [ ]:
print(f'T test: {ttest_ind(df1["bias_prior_corrected"], df2["bias_prior_corrected"])}')
print(f'Ranksums: {ranksums(df1["bias_prior_corrected"], df2["bias_prior_corrected"])}')
print(f'Exact Monte Carlo permutation test: {exact_mc_perm_test(df1["bias_prior_corrected"], df2["bias_prior_corrected"])}')